# Compiling Circuits for EeroQ via Cirq

## Import Requirements

This tutorial will showcase how to compile a circuit for EeroQ hardware using the ```cirq-superstaq``` client. 

In [1]:
# Required imports
try:
    import cirq
    import cirq_superstaq as css
except ImportError:
    print("Installing cirq-superstaq...")
    %pip install --quiet 'cirq-superstaq[examples]'
    print("Installed cirq-superstaq.")
    print("You may need to restart the kernel to import newly installed packages.")
    import cirq
    import cirq_superstaq as css

try:
    import qiskit
    import qiskit_superstaq as qss
except ImportError:
    print("Installing qiskit-superstaq...")
    %pip install --quiet 'qiskit-superstaq[examples]'
    print("Installed qiskit-superstaq.")
    print("You may need to restart the kernel to import newly installed packages.")
    import qiskit
    import qiskit_superstaq as qss

# Optional imports
import numpy as np
import os  # Used if setting a token as an environment variable

%load_ext autoreload
%autoreload 2

To interface Superstaq via Cirq, we must first instantiate a service provider in ```cirq-superstaq``` with ```Service()```. We then supply a Superstaq API key (which you can get from https://superstaq.infleqtion.com) by either providing the API key as an argument of Service, i.e., ```css.Service(api_key="token")```, or by setting it as an environment variable. (see more details [here](https://superstaq.readthedocs.io/en/latest/get_started/basics/basics_css.html#Set-up-access-to-Superstaq%E2%80%99s-API)).

In [2]:
service = css.Service()

# EeroQ Gates

In [3]:
dd_gate = css.DDPowGate(exponent=1)
cirq.Circuit(dd_gate.on(cirq.q(0), cirq.q(1)))

0: ───DD───
      │
1: ───DD───

In [4]:
cirq.unitary(dd_gate)

array([[ 0.-1.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j, -1.+0.j,  0.+0.j],
       [ 0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.-1.j]])

Below is the EeroQ protocol for a CZ Gate.

![title](images/DD_gate.png)

In [5]:
circuit = cirq.Circuit(cirq.CZ(cirq.q(0), cirq.q(1)))
compiled_circuit = service.compile(circuit, "eeroq_wonderlake_qpu").circuit
compiled_circuit

0+: ───────│──────────────────────────────
           │
0-: ───X───│───DD────────S──────DD────────
           │   │                │
1+: ───────│───┼─────────Z──────┼─────────
           │   │                │
1-: ───X───│───DD^0.25───S^-1───DD^0.25───

In [6]:
def compute_unitary(circuit: cirq.Circuit):
    """Computes the n*n unitary of a 2n electron EeroQ circuit"""
    unitary = cirq.unitary(circuit[1:]).reshape((4,) * cirq.num_qubits(circuit))
    mat = unitary[tuple(slice(1, 3) for _ in range(cirq.num_qubits(circuit)))]
    dim = round(np.sqrt(mat.size))
    mat = mat.reshape(dim, dim)
    return mat

In [7]:
mat = compute_unitary(compiled_circuit)
mat/mat[0][0]

array([[ 1.-0.00000000e+00j, -0.+0.00000000e+00j, -0.+0.00000000e+00j,
        -0.+0.00000000e+00j],
       [-0.+0.00000000e+00j,  1.+7.85046229e-17j, -0.+0.00000000e+00j,
        -0.+0.00000000e+00j],
       [-0.+0.00000000e+00j, -0.+0.00000000e+00j,  1.+7.85046229e-17j,
        -0.+0.00000000e+00j],
       [-0.+0.00000000e+00j, -0.+0.00000000e+00j, -0.+0.00000000e+00j,
        -1.-0.00000000e+00j]])

In [8]:
cirq.allclose_up_to_global_phase(cirq.unitary(circuit), mat)

True

In [9]:
circuit += cirq.measure(cirq.q(0), cirq.q(1))
job = service.create_job(circuit, target= "eeroq_wonderlake_qpu", repetitions = 10, method="dry-run")

In [10]:
job.counts()

[{'00': 10}]

# Circuit Compilation

In [14]:
qubits = cirq.LineQubit.range(2)
circuit = cirq.Circuit(cirq.H(qubits[0]), cirq.CNOT(qubits[0], qubits[1]), cirq.measure(*qubits))
circuit

0: ───H───@───M───
          │   │
1: ───────X───M───

In [15]:
compiled_circuit = service.compile(circuit, "eeroq_wonderlake_qpu").circuit
compiled_circuit

0+: ───────│───S^-1───│───DD────────S──────│─────────────Z────────────────│───DD────────Z──────│───M('q(0),q(1)')───
           │          │   │                │                              │   │                │   │
0-: ───X───│──────────│───DD^-0.5──────────│───DD────────S──────DD────────│───DD───────────────│───M────────────────
           │          │                    │   │                │         │                    │   │
1+: ───────│───S──────│───DD────────S^-1───│───┼─────────S^-1───┼─────────│───DD────────S^-1───│───M────────────────
           │          │   │                │   │                │         │   │                │   │
1-: ───X───│──────────│───DD^-0.5──────────│───DD^0.25───S^-1───DD^0.25───│───DD^-0.5──────────│───M────────────────

In [16]:
gate_domain = {
    cirq.X: 1,
    cirq.Y: 1,
    cirq.Z: 1,
    cirq.S: 1,
    cirq.T: 1,
    cirq.H: 1,
    cirq.rx(1.23): 1,
    cirq.ry(2.34): 1,
    cirq.CZ: 2,
    cirq.CX: 2,
    cirq.CX**0.5: 2,
    cirq.SWAP: 2,
    cirq.ISWAP: 2,
    css.ZZSwapGate(1.23): 2,
    css.Barrier(3): 3,
}

In [17]:
n, depth, op_density = (4, 8, 0.8)
qubits = cirq.LineQubit.range(n)
circuit = cirq.testing.random_circuit(qubits, depth, op_density, gate_domain=gate_domain)
circuit

0: ───Y───X───iSwap───Ry(0.745π)───────────H────────────────@───
          │   │                                             │
1: ───────@───iSwap────────────────X^0.5────────────────Y───X───
                                   │
2: ───────X───────────S────────────┼───────Rx(0.392π)───T───────
                                   │
3: ────────────────────────────────@────────────────────────────

In [15]:
circuit.insert(depth // 2, css.barrier(*qubits))
compiled_circuit = service.compile(circuit, "eeroq_wonderlake_qpu").circuit
compiled_circuit

┌──────────────┐
0+: ───────│───────│──────────────────────│──────────────────────────────│───────────────│──────────────────────────────│──────────────────────Z^-0.75─────────────│──────────────────────────────│────────────────────│───────│──────────────────────────────│───────────────────│──────────────────────────────│────────────────────│──────────────────────────────│─────────────────
           │       │                      │                              │               │                              │                                          │                              │                    │       │                              │                   │                              │                    │                              │
0-: ───X───│───────│──────────────────────│──────────────────────────────│───────────────│──────────────────────────────│────DD────────────────S^-1──────DD────────│──────────────────────────────│────────────────────│───────│──────────────────────────────│───────────────────│──────────────────────────────│────────────────────│──────────────────────────────│─────────────────
           │       │                      │                              │               │                              │    │                           │         │                              │                    │       │                              │                   │                              │                    │                              │
1+: ───────│───Z───│───DD──────────Z──────│──────────────────────────────│───────────────│─────────────Z────────────────│────┼──────DD─────────Z─────────┼─────────│─────────────Z────────────────│───DD────────S──────│───S───│───DD────────S^-1─────────────│───────────────────│─────────────Z────────────────│────────────────────│──────────────────────────────│─────────────────
           │       │   │                  │                              │               │                              │    │      │                    │         │                              │   │                │       │   │                          │                   │                              │                    │                              │
1-: ───X───│───────│───DD^-0.108──────────│──────────────────────────────│───────────────│───DD────────S──────DD────────│────┼──────DD^-0.5──────────────┼─────────│───DD────────S──────DD────────│───DD^-0.5──────────│───────│───DD^-0.5────────────────────│───────────────────│───DD────────S──────DD────────│────────────────────│──────────────────────────────│─────────────────
           │       │                      │                              │               │   │                │         │    │                           │         │   │                │         │                    │       │                              │                   │   │                │         │                    │                              │
2+: ───────│───────│──────────────────────│─────────────S^-1─────────────│───DD──────────│───┼─────────S──────┼─────────│────┼───────────────────────────┼─────────│───┼────────────────┼─────────│────────────────────│───────│─────────────Z────────────────│───────────────────│───┼────────────────┼─────────│────────────────────│──────────────────────────────│─────────────────
           │       │                      │                              │   │           │   │                │         │    │                           │         │   │                │         │                    │       │                              │                   │   │                │         │                    │                              │
2-: ───X───│───────│──────────────────────│───DD────────S──────DD────────│───DD^-0.745───│───┼────────────────┼─────────│────DD^0.25───────────S─────────DD^0.25───│───┼────────────────┼─────────│────────────────────│───────│───DD────────S^-1───DD────────│───────────────────│───┼────────────────┼─────────│──

In [16]:
mat = compute_unitary(compiled_circuit)
cirq.testing.assert_allclose_up_to_global_phase(cirq.unitary(circuit), mat, atol=1e-8)

In [2]:
service = qss.superstaq_provider.SuperstaqProvider(remote_host="https://127.0.0.1:5000")

In [78]:
from qiskit.circuit.random import random_circuit
qc = random_circuit(2, 1, measure=False)

In [79]:
qc.draw()

┌────────────┐
q_0: ┤ Rz(3.4036) ├
     ├────────────┤
q_1: ┤ Rx(4.4067) ├
     └────────────┘

In [80]:
backend = service.get_backend("eeroq_wonderlake_qpu")

In [81]:
cc = backend.compile(qc).circuit

/home/palash/cq-env/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [82]:
cc.draw()

░ ┌─────────────┐ ░                      
00+: ──────░─┤ Rz(-2.8796) ├─░──────────────────────
     ┌───┐ ░ └─────────────┘ ░                      
00-: ┤ X ├─░─────────────────░──────────────────────
     └───┘ ░      ┌───┐      ░ ┌──────────────┐┌───┐
01+: ──────░──────┤ Z ├──────░─┤0             ├┤ Z ├
     ┌───┐ ░      └───┘      ░ │  Dd(-1.8765) │└───┘
01-: ┤ X ├─░─────────────────░─┤1             ├─────
     └───┘ ░                 ░ └──────────────┘     
02+: ───────────────────────────────────────────────
                                                    
02-: ───────────────────────────────────────────────
                                                    
03+: ───────────────────────────────────────────────
                                                    
03-: ───────────────────────────────────────────────
                                                    
04+: ───────────────────────────────────────────────
                                                    
04-: ───────────────────────────────────────────────
                                                    
05+: ───────────────────────────────────────────────
                                                    
05-: ───────────────────────────────────────────────
                                                    
06+: ───────────────────────────────────────────────
                                                    
06-: ───────────────────────────────────────────────
                                                    
07+: ───────────────────────────────────────────────
                                                    
07-: ───────────────────────────────────────────────
                                                    
08+: ───────────────────────────────────────────────
                                                    
08-: ───────────────────────────────────────────────
                                                    
09+: ───────────────────────────────────────────────
                                                    
09-: ───────────────────────────────────────────────

In [86]:
def compute_qiskit_unitary(qc: qiskit.QuantumCircuit, n):
    """Computes the n*n unitary of a 2n electron EeroQ circuit"""
    instructions = qc[n:]
    new_circuit = qiskit.QuantumCircuit(qc.qubits, qc.clbits)
    
    for instruction, qargs, cargs in instructions:
        new_circuit.append(instruction, qargs, cargs)
    
    qc = remove_idle_wires(new_circuit)
    mat = qiskit.quantum_info.Operator(qc).to_matrix().reshape((4,) * qc.num_qubits)
    print(qc)
    mat = mat[tuple(slice(1, 3) for _ in range(qc.num_qubits))]
    dim = round(np.sqrt(mat.size))
    mat = mat.reshape(dim, dim)
    return mat

mat = compute_qiskit_unitary(cc, 2)
mat

    ░ ┌─────────────┐ ░                      
0: ─░─┤ Rz(-2.8796) ├─░──────────────────────
    ░ └─────────────┘ ░                      
1: ─░─────────────────░──────────────────────
    ░      ┌───┐      ░ ┌──────────────┐┌───┐
2: ─░──────┤ Z ├──────░─┤0             ├┤ Z ├
    ░      └───┘      ░ │  Dd(-1.8765) │└───┘
3: ─░─────────────────░─┤1             ├─────
    ░                 ░ └──────────────┘     


array([[-0.42707381-0.40882073j,  0.        +0.j        ,
         0.5577091 -0.58260977j,  0.        +0.j        ],
       [ 0.        +0.j        ,  0.51839478+0.28424158j,
         0.        +0.j        , -0.38775949+0.70718891j],
       [ 0.5577091 -0.58260977j,  0.        +0.j        ,
        -0.42707381-0.40882073j,  0.        +0.j        ],
       [ 0.        +0.j        , -0.38775949+0.70718891j,
         0.        +0.j        ,  0.51839478+0.28424158j]])

In [87]:
init_mat = qiskit.quantum_info.Operator(qc).to_matrix()
init_mat

array([[ 0.07723258+0.58614124j,  0.        +0.j        ,
        -0.79960795+0.1053599j ,  0.        +0.j        ],
       [ 0.        +0.j        ,  0.07723258-0.58614124j,
         0.        +0.j        ,  0.79960795+0.1053599j ],
       [-0.79960795+0.1053599j ,  0.        +0.j        ,
         0.07723258+0.58614124j,  0.        +0.j        ],
       [ 0.        +0.j        ,  0.79960795+0.1053599j ,
         0.        +0.j        ,  0.07723258-0.58614124j]])

In [90]:
# Might need to swap endianness
cirq.equal_up_to_global_phase(mat, init_mat)

False

In [21]:
def count_gates(qc: qiskit.QuantumCircuit):
    gate_count = { qubit: 0 for qubit in qc.qubits }
    for gate in qc.data:
        for qubit in gate.qubits:
            gate_count[qubit] += 1
    return gate_count

def remove_idle_wires(qc: qiskit.QuantumCircuit):
    qc_out = qc.copy()
    gate_count = count_gates(qc_out)
    for qubit, count in gate_count.items():
        if count == 0:
            qc_out.qubits.remove(qubit)
    return qc_out


In [23]:
remove_idle_wires(cc).draw()

░ ┌─────────────┐ ░  ┌───────────┐   ┌────────────┐  ░    ┌─┐
00+: ──────░─┤ Rz(-1.3351) ├─░──┤0          ├───┤ Rz(1.3351) ├──░────┤M├
     ┌───┐ ░ └─────────────┘ ░  │  Dd(-π/2) │   └────────────┘  ░    └╥┘
00-: ┤ X ├─░─────────────────░──┤1          ├───────────────────░─────╫─
     └───┘ ░ ┌─────────────┐ ░ ┌┴───────────┴─┐┌──────────────┐ ░ ┌─┐ ║ 
01+: ──────░─┤ Rz(0.79118) ├─░─┤0             ├┤ Rz(-0.79118) ├─░─┤M├─╫─
     ┌───┐ ░ └─────────────┘ ░ │  Dd(-1.0005) │└──────────────┘ ░ └╥┘ ║ 
01-: ┤ X ├─░─────────────────░─┤1             ├─────────────────░──╫──╫─
     └───┘ ░                 ░ └──────────────┘                 ░  ║  ║ 
c: 2/══════════════════════════════════════════════════════════════╩══╩═
                                                                   1  0

# Circuit Simulation

In [18]:
circuit+= cirq.measure(*cirq.LineQubit.range(4))
circuit

0: ───Y───X───iSwap───Ry(0.745π)───────────H────────────────@───M───
          │   │                                             │   │
1: ───────@───iSwap────────────────X^0.5────────────────Y───X───M───
                                   │                            │
2: ───────X───────────S────────────┼───────Rx(0.392π)───T───────M───
                                   │                            │
3: ────────────────────────────────@────────────────────────────M───

In [19]:
service.run([circuit], 1000, "eeroq_wonderlake_qpu", method="dry-run")

[q(0),q(1),q(2),q(3)=0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

In [21]:
service.run([circuit], 1000, "eeroq_wonderlake_qpu", method="noise-sim", noise=0.01)

[q(0),q(1),q(2),q(3)=0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000011111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

In [24]:
help(service.run)

Help on method run in module cirq_superstaq.service:

run(circuits: 'cirq.Circuit | Sequence[cirq.Circuit]', repetitions: 'int', target: 'str | None' = None, param_resolver: 'cirq.ParamResolver' = cirq.ParamResolver({}), method: 'str | None' = None, **kwargs: 'Any') -> 'cirq.ResultDict | list[cirq.ResultDict]' method of cirq_superstaq.service.Service instance
    Runs circuit(s) on the Superstaq API and returns the result(s) as `cirq.ResultDict`.
    
    `service.create_job()` or `service.get_counts()` instead.
    
    Args:
        circuits: The circuit(s) to run.
        repetitions: The number of times to run the circuit(s).
        target: Where to run the job.
        param_resolver: A `cirq.ParamResolver` to resolve parameters in `circuits`.
        method: Execution method.
        kwargs: Other optimization and execution parameters.
    
    Returns:
        The `cirq.ResultDict` object(s) from running the circuit(s).

